# Applying Advanced Transformations

## The Data

You will be working with a heavily modified version of the Superheroes dataset from Kaggle.

Source: https://www.kaggle.com/datasets/claudiodavi/superhero-set

The dataset includes two csv's:

- superhero_info.csv: https://docs.google.com/spreadsheets/d/e/2PACX-1vS1ZstYLwFgwhZnqDsPjtnlHYhJp_cmW55J8JD5mym0seRsaem3px7QBtuFF0LiI7z1PLCkVKAkdO7J/pub?output=csv
  - Contains Name, Publisher, Demographic Info, and Body measurements.

- superhero_powers.csv: https://docs.google.com/spreadsheets/d/e/2PACX-1vSzdWOBaXOoz52vPmCFV5idNlDBohLY1Lsbc1IfZIZQ7cV_aNB2wYBfhF49uE1TaO1B5MQCGWiNrFfd/pub?output=csv
  - Contains Hero name and list of powers

In [58]:
# imports
import pandas as pd
import json

In [59]:
# load first csv
info = pd.read_csv('Data/superhero_info - superhero_info.csv')

# check
info.info()
info.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 463 entries, 0 to 462
Data columns (total 8 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Hero|Publisher  463 non-null    object
 1   Gender          463 non-null    object
 2   Race            463 non-null    object
 3   Alignment       463 non-null    object
 4   Hair color      463 non-null    object
 5   Eye color       463 non-null    object
 6   Skin color      463 non-null    object
 7   Measurements    463 non-null    object
dtypes: object(8)
memory usage: 29.1+ KB


,Hero|Publisher,Gender,Race,Alignment,Hair color,Eye color,Skin color,Measurements
0,A-Bomb|Marvel Comics,Male,Human,good,No Hair,yellow,Unknown,"{'Height': '203.0 cm', 'Weight': '441.0 kg'}"
1,Abe Sapien|Dark Horse Comics,Male,Icthyo Sapien,good,No Hair,blue,blue,"{'Height': '191.0 cm', 'Weight': '65.0 kg'}"
2,Abin Sur|DC Comics,Male,Ungaran,good,No Hair,blue,red,"{'Height': '185.0 cm', 'Weight': '90.0 kg'}"
3,Abomination|Marvel Comics,Male,Human / Radiation,bad,No Hair,green,Unknown,"{'Height': '203.0 cm', 'Weight': '441.0 kg'}"
4,Absorbing Man|Marvel Comics,Male,Human,bad,No Hair,blue,Unknown,"{'Height': '193.0 cm', 'Weight': '122.0 kg'}"


In [60]:
# load second csv
powers = pd.read_csv('Data/superhero_powers - superhero_powers.csv')

# check
powers.info()
powers.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 667 entries, 0 to 666
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   hero_names  667 non-null    object
 1   Powers      667 non-null    object
dtypes: object(2)
memory usage: 10.5+ KB


,hero_names,Powers
0,3-D Man,"Agility,Super Strength,Stamina,Super Speed"
1,A-Bomb,"Accelerated Healing,Durability,Longevity,Super..."
2,Abe Sapien,"Agility,Accelerated Healing,Cold Resistance,Du..."
3,Abin Sur,Lantern Power Ring
4,Abomination,"Accelerated Healing,Intelligence,Super Strengt..."


## The Task

### I. Clean the files and combine them into one final DataFrame.

- This dataframe should have the following columns:
  - Hero (Just the name of the Hero)
  - Publisher
  - Gender
  - Eye color
  - Race
  - Hair color
  - Height (numeric)
  - Skin color
  - Alignment
  - Weight (numeric)
  - Plus, one-hot-encoded columns for every power that appears in the dataset. E.g.:
    - Agility
    - Flight
    - Superspeed
    - etc.

Hint: There is a space in "100 kg" or "52.5 cm"

#### 'Hero' and 'Publisher'

In [61]:
# split the Hero|Publisher column into two columns
info[['Hero', 'Publisher']] = info['Hero|Publisher'].str.split('|', expand = True)

# check
info.head(2)

,Hero|Publisher,Gender,Race,Alignment,Hair color,Eye color,Skin color,Measurements,Hero,Publisher
0,A-Bomb|Marvel Comics,Male,Human,good,No Hair,yellow,Unknown,"{'Height': '203.0 cm', 'Weight': '441.0 kg'}",A-Bomb,Marvel Comics
1,Abe Sapien|Dark Horse Comics,Male,Icthyo Sapien,good,No Hair,blue,blue,"{'Height': '191.0 cm', 'Weight': '65.0 kg'}",Abe Sapien,Dark Horse Comics


In [62]:
# drop original 'Hero|Publisher' column
info = info.drop(columns = ['Hero|Publisher'])

# check
info.head(2)

,Gender,Race,Alignment,Hair color,Eye color,Skin color,Measurements,Hero,Publisher
0,Male,Human,good,No Hair,yellow,Unknown,"{'Height': '203.0 cm', 'Weight': '441.0 kg'}",A-Bomb,Marvel Comics
1,Male,Icthyo Sapien,good,No Hair,blue,blue,"{'Height': '191.0 cm', 'Weight': '65.0 kg'}",Abe Sapien,Dark Horse Comics


#### 'Height' and 'Weight'

In [63]:
# check type of 'Measurements' column
measurement = info.loc[0, 'Measurements']
print(type(measurement))
print(measurement)

<class 'str'>
{'Height': '203.0 cm', 'Weight': '441.0 kg'}


In [64]:
# convert strings in dictionary to using double instead of
# single quotes for json.loads compatibility
info['Measurements'] = info['Measurements'].str.replace("'", '"')

# check
measurement = info.loc[0, 'Measurements']
print(measurement)

{"Height": "203.0 cm", "Weight": "441.0 kg"}


In [65]:
# use .apply with json.loads to convert strings to dicts
info['Measurements'] = info['Measurements'].apply(json.loads)

# check
measurement = info.loc[0, 'Measurements']
print(type(measurement))

<class 'dict'>


In [66]:
# unpack 'Measurements' column (dictionaries) into separate
# columns for each dictionary item
height_weight = info['Measurements'].apply(pd.Series)

# check
height_weight

,Height,Weight
0,203.0 cm,441.0 kg
1,191.0 cm,65.0 kg
2,185.0 cm,90.0 kg
3,203.0 cm,441.0 kg
4,193.0 cm,122.0 kg
...,...,...
458,183.0 cm,83.0 kg
459,165.0 cm,52.0 kg
460,66.0 cm,17.0 kg
461,170.0 cm,57.0 kg


In [67]:
# concatenate new columns to original df
info = pd.concat((info, height_weight), axis = 1)

# check
info.head(2)

,Gender,Race,Alignment,Hair color,Eye color,Skin color,Measurements,Hero,Publisher,Height,Weight
0,Male,Human,good,No Hair,yellow,Unknown,"{'Height': '203.0 cm', 'Weight': '441.0 kg'}",A-Bomb,Marvel Comics,203.0 cm,441.0 kg
1,Male,Icthyo Sapien,good,No Hair,blue,blue,"{'Height': '191.0 cm', 'Weight': '65.0 kg'}",Abe Sapien,Dark Horse Comics,191.0 cm,65.0 kg


In [68]:
# drop original column
info = info.drop(columns = ['Measurements'])

# check
info.head(2)

,Gender,Race,Alignment,Hair color,Eye color,Skin color,Hero,Publisher,Height,Weight
0,Male,Human,good,No Hair,yellow,Unknown,A-Bomb,Marvel Comics,203.0 cm,441.0 kg
1,Male,Icthyo Sapien,good,No Hair,blue,blue,Abe Sapien,Dark Horse Comics,191.0 cm,65.0 kg


In [69]:
# change 'Height' and 'Weight' column names
info.rename(columns = {'Height': 'Height (cm)',
                      'Weight': 'Weight (kg)'}, 
            inplace = True)

# check
info.head(2)

,Gender,Race,Alignment,Hair color,Eye color,Skin color,Hero,Publisher,Height (cm),Weight (kg)
0,Male,Human,good,No Hair,yellow,Unknown,A-Bomb,Marvel Comics,203.0 cm,441.0 kg
1,Male,Icthyo Sapien,good,No Hair,blue,blue,Abe Sapien,Dark Horse Comics,191.0 cm,65.0 kg


In [70]:
# take off 'cm' and 'kg' from cell values in 'Height' and 'Weight'
to_replace = [' cm', ' kg']

for char in to_replace:
    info['Height (cm)'] = info['Height (cm)'].str.replace(char, 
                                                          '', 
                                                          regex = False)
    info['Weight (kg)'] = info['Weight (kg)'].str.replace(char, 
                                                          '', 
                                                          regex = False)
    
# check
info[['Height (cm)', 'Weight (kg)']].head(2)

,Height (cm),Weight (kg)
0,203.0,441.0
1,191.0,65.0


In [71]:
# convert 'Height (cm)' and 'Weight (kg)' columns from object-
# types to numeric

info['Height (cm)'] = info['Height (cm)'].astype(float)
info['Weight (kg)'] = info['Weight (kg)'].astype(float)

# check
info.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 463 entries, 0 to 462
Data columns (total 10 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Gender       463 non-null    object 
 1   Race         463 non-null    object 
 2   Alignment    463 non-null    object 
 3   Hair color   463 non-null    object 
 4   Eye color    463 non-null    object 
 5   Skin color   463 non-null    object 
 6   Hero         463 non-null    object 
 7   Publisher    463 non-null    object 
 8   Height (cm)  463 non-null    float64
 9   Weight (kg)  463 non-null    float64
dtypes: float64(2), object(8)
memory usage: 36.3+ KB


#### Powers

In [72]:
# check type of powers['Powers'] column
power = powers.loc[0, 'Powers']
print(type(power))
print(power)

<class 'str'>
Agility,Super Strength,Stamina,Super Speed


In [74]:
# replace single quotes with double quotes
powers['powers_split'] = powers['Powers'].str.replace("'", '"')

# apply json.loads
powers['powers_split'] = powers['powers_split'].apply(json.loads)

# check
powers['powers_split'].head()

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [ ]:
# combine two dataframes on info['Hero'] and powers['hero_names']

### II. Use your combined DataFrame to answer the following questions.

1. Compare the average weight of super powers who have Super Speed to those who do not.

2. What is the average height of heroes for each publisher?